In [104]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import math

In [105]:
def sigmoid(data):
    return 1 / (1 + np.exp(-data))

def softmax(w, t=1.0):
    e = np.exp(w / t)
    dist = e / np.sum(e)
    return dist

def ReLU(x):
    return np.where(x > 0, 1.0, 0.0)

In [106]:
IMAGE_PATH = '/home/aishwarya/CSIS/SMAI/SMAI_assig/a-6/test.png'

Filter_Row_Dim = 5
Filter_Col_Dim = 5

FC_Convolution_Filter_Count = 120
POOLING_DIM = 2

def create_filters(layer_count):
    filter_depth = 0
    if layer_count == 1:
        filter_depth = 6
    elif layer_count == 2:
        filter_depth = 16
    elif layer_count == 3:
        filter_depth = FC_Convolution_Filter_Count
    else:
        print("Invalid layer count")

    filter_matrix = np.random.randn(Filter_Row_Dim, Filter_Col_Dim, filter_depth)
    return filter_matrix

def convolve_oper(data, filter):  
    single_convolve = np.multiply(data, filter)
    return single_convolve

def convolution(data, filter):
    filter_row, filter_col, num_filters = filter.shape
    data_row, data_col, data_channels = data.shape
    conv_result_dim = data_row - filter_row + 1
    ans=0
    convolve_result = np.zeros((conv_result_dim, conv_result_dim, num_filters))

    # Breaking the data into sub-matrices of filter size and calling convolve_oper
    for filter_index in range(num_filters):        
        for x_index in range(conv_result_dim):
            for y_index in range(conv_result_dim):
                ans=convolve_oper(data[x_index: x_index + filter_row, y_index: y_index + filter_col, filter_index % data_channels],filter[:, :, filter_index])
                convolve_result[x_index][y_index][filter_index] = ans.sum()
#         im = Image.fromarray(convolve_result)
#         nm='test'+filter_index+'.png'
#         im.save(nm)

    return convolve_result

def imgshow(x,name):
    #x = x.transpose(2,0,1) 
    img = Image.fromarray(x,'RGB')
    img = img.resize((300,300))
    img.save(name)
#     img.show()      

def calc_max_pooling(data, sze=2):#, pool_size):
    data_row, data_col, result_layer = data.shape
    result_row= data_row // sze
    result_col= data_col // sze

    max_pool_result = np.zeros((result_row, result_col, result_layer))

    for l in range(result_layer):
        for r in range(0, data_row, 2):
            for c in range(0, data_col, 2):
                max_pool_result[r // sze][c // sze][l] = (data[r:r + sze, c:c + sze, l]).max()

    return max_pool_result

In [112]:
# if __name__ == '__main__':
def CNN(act_fun="sigmoid",pooling_type="max_pooling"):
    # Lenet Architectue
    # INPUT => CONV => RELU => POOL => CONV => RELU => POOL => FC(Conv) => RELU => FC

    image = Image.open(IMAGE_PATH)
    image = image.resize((32,32), Image.ANTIALIAS)
    image.save('resized_img_32x32.png')
    
    image_array = np.array(image)
    print "Image new Dimensions: ", image_array.shape

    # -------------------------- First Convolution Block -------------------------------------------
    # Creating first layer filter
    # apply convolution on original image with 6 diffrent filters
    print '\n\nFirst convolution:'
    filter_matrix = create_filters(1)
    print "Filter Dimensions: ", filter_matrix.shape
    
    imgshow(filter_matrix,'1_filter_matrix.png')

    # Convolution at first layer
    convolve_result = convolution(image_array, filter_matrix)
#     print 'type========',convolve_result.type
    print "Convolution Result Dimensions:", convolve_result.shape
    imgshow(convolve_result,'1_conv_result.png')
    
    if act_fun=="ReLU":
        # Applying ReLU activation function
        relu_result = ReLU(convolve_result)
        print "ReLU Result Dimensions:", relu_result.shape
        imgshow(relu_result,'1_relu_result.png')
    elif act_fun=="sigmoid":
        relu_result = sigmoid(convolve_result)
        print "sigmoid Result Dimensions:", relu_result.shape
        imgshow(relu_result,'1_sigmoid_result.png')


    if pooling_type=="max_pooling":
        # MaxPooling at the first convolution block
        pool_result = calc_max_pooling(relu_result, POOLING_DIM)
        imgshow(pool_result,'1_maxpool_result.png')
    print pool_result.shape , 'New image shape after 1st pooling'

    # --------------Second Convolution Block-----------------------------------------
    print '\n\nSecond convolution:'
    filter_matrix = create_filters(2)
    print "Filter Dimensions (2nd Block): ", filter_matrix.shape

    imgshow(filter_matrix,'2_filter_matrix.png')

    convolve_result = convolution(pool_result, filter_matrix)
    print "Convolution Result Dimensions (2nd Block):", convolve_result.shape

    imgshow(convolve_result,'2_conv_result.png')

    if act_fun=="ReLU":
        # Applying ReLU activation function
        relu_result = ReLU(convolve_result)
        print "ReLU Result Dimensions:", relu_result.shape
        imgshow(relu_result,'2_relu_result.png')
    elif act_fun=="sigmoid":
        relu_result = sigmoid(convolve_result)
        print "sigmoid Result Dimensions:", relu_result.shape
        imgshow(relu_result,'2_sigmoid_result.png')

        

    if pooling_type=="max_pooling":
        # MaxPooling at the first convolution block
        pool_result = calc_max_pooling(relu_result, POOLING_DIM)
        imgshow(pool_result,'2_maxpool_result.png')

    print pool_result.shape , 'New image shape after 2nd pooling'

    # -------------------- Convolution to give Fully Connected Layer-----------------------
    print '\n\nFully Connected Layer-----------------------'
    
    print '[forward propagation]\nConvolution to give Fully Connected Layer:-----------'
    
    input_mat=np.array(pool_result.flatten().reshape(1, len(pool_result.flatten())))
    print 'current shape-',input_mat.shape

    # -----------first layer-----------
    weight_mat=np.random.uniform(low=-1, high=1, size=(400,120) )
    output_layer1=np.matmul(input_mat, weight_mat)
    for i in range( len (output_layer1[0]) ):
        output_layer1[0][i]=sigmoid(output_layer1[0][i])

    print '\noutput layer 1 shape-',output_layer1.shape

    # -----------second layer-----------
    input_mat=output_layer1
    weight_mat=np.random.uniform(low=-1, high=1, size=(120,84) )
    output_layer2=np.matmul(input_mat, weight_mat)
    for i in range( len (output_layer2[0]) ):
        output_layer2[0][i]=sigmoid(output_layer2[0][i])

    print '\noutput layer 2 shape-',output_layer2.shape

    # -----------third layer-----------
    input_mat=output_layer2
    weight_mat=np.random.uniform(low=-1, high=1, size=(84,10) )
    output_layer3=np.matmul(input_mat, weight_mat)
    for i in range( len (output_layer3[0]) ):
        output_layer3[0][i]=sigmoid(output_layer3[0][i])

    print '\noutput layer 3 shape-',output_layer3.shape
    print '\noutput layer 3 -',output_layer3

In [113]:
act_fun="ReLU"
CNN(act_fun,pooling_type="max_pooling")

Image new Dimensions:  (32, 32, 4)


First convolution:
Filter Dimensions:  (5, 5, 6)
Convolution Result Dimensions: (28, 28, 6)
ReLU Result Dimensions: (28, 28, 6)
(14, 14, 6) New image shape after 1st pooling


Second convolution:
Filter Dimensions (2nd Block):  (5, 5, 16)
Convolution Result Dimensions (2nd Block): (10, 10, 16)
ReLU Result Dimensions: (10, 10, 16)
(5, 5, 16) New image shape after 2nd pooling


Fully Connected Layer-----------------------
[forward propagation]
Convolution to give Fully Connected Layer:-----------
current shape- (1, 400)

output layer 1 shape- (1, 120)

output layer 2 shape- (1, 84)

output layer 3 shape- (1, 10)

output layer 3 - [[ 0.01504269  0.1405255   0.91077195  0.96774205  0.29237084  0.9555129
   0.01359112  0.01383743  0.04250676  0.78805737]]
